# 1. Dependencies

In [ ]:
!apt install libnvrtc8.0
!pip uninstall -y mxnet-cu80 && pip install mxnet-cu80==1.1.0
!pip install turicreate
import turicreate as tc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libnvrtc8.0
E: Couldn't find any package by glob 'libnvrtc8.0'
E: Couldn't find any package by regex 'libnvrtc8.0'
  Using cached mxnet_cu80-1.1.0-py2.py3-none-manylinux1_x86_64.whl (274.9 MB)
  Using cached numpy-1.13.3.zip (5.0 MB)
  Using cached graphviz-0.8.1-py2.py3-none-any.whl (17 kB)
  Using cached requests-2.18.4-py2.py3-none-any.whl (88 kB)
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
  ERROR: Failed cleaning build dir for numpy
Failed to build numpy
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
    Running setup.py install for numpy ... error
  Rolling back un

In [ ]:
# !pip install turicreate --user
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import sqrt
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()
# import turicreate as tc
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
from sklearn import preprocessing

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Data Preparation
## 2.1 Read Data

In [ ]:
# customer, article, transaction;
# article_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv",header=0)
customer_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/567_hw/data/customers.csv",header=0)
transaction_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/567_hw/data/transactions_train.csv",header=0)



## 2.2 Create data with user, item, and target field

In [ ]:


# Get Customer_Article table to show Transaction Record;
customer_article = transaction_df.groupby(["customer_id", "article_id"])[["article_id"]].count()
customer_article.columns = ["purchase_count"]
customer_article = customer_article.reset_index()


# customer_article = customer_article.sort_values(["customer_id", "purchase_count"], ascending=False)
# customer_article = customer_article.groupby("customer_id").head(10)
# customer_article = customer_article.groupby("customer_id")["article_id"].apply(list)
# customer_article = customer_article[:1000]
print(customer_article.shape)

(27306439, 3)


## 2.3 create dummy

In [ ]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
customer_article_dummy = create_data_dummy(customer_article)
customer_article_dummy.head()

,customer_id,article_id,purchase_count,purchase_dummy
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023,1,1
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006,2,1
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043,1,1
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,607642008,1,1
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001,1,1


## 2.4 Normalized data

In [ ]:

# df_matrix = pd.pivot_table(customer_article, values='purchase_count', index='customer_id', columns='article_id')
# df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
# df_matrix_norm
# def normalize_data(data):
#     # df_matrix = pd.pivot_table(data, values='purchase_count', index='customer_id', columns='article_id')
#     # df_matrix = data.set_index(['customer_id', 'article_id'])['purchase_count'].unstack('article_id')
#     df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
#     d = df_matrix_norm.reset_index()
#     d.index.names = ['scaled_purchase_freq']
#     return pd.melt(d, id_vars=['customer_id'], value_name='scaled_purchase_freq').dropna()

# customer_article_norm = normalize_data(customer_article)
# customer_article_norm.head()

# 3. Rank articles with the Popularity

In [ ]:
# transaction_df['article_id'].value_counts()[:12]
# popular_articles = list(transaction_dev['article_id'].value_counts().index[:12])

## 3.1 Splite data Function

In [ ]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

train_data, test_data = split_data(customer_article)
train_data_dummy, test_data_dummy = split_data(customer_article_dummy)
# train_data_norm, test_data_norm = split_data(customer_article_norm)

In [ ]:
# constant variables to define field names include:
user_id = 'customer_id'
item_id = 'article_id'
users_to_recommend = list(customer_df[user_id])
n_rec = 12 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                        user_id=user_id, 
                                                        item_id=item_id, 
                                                        target=target, 
                                                        similarity_type='pearson')

    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [ ]:
# #1.
name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)
popularity
# #2.
# name = 'popularity'
# target = 'purchase_dummy'
# pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)
#3.
# name = 'popularity'
# target = 'scaled_purchase_freq'
# pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 21845151 observations with 1325030 users and 103491 items.

Data prepared in: 8.29274s

21845151 observations to process; with 103491 unique items.

recommendations finished on 1000/1371980 queries. users per second: 8278.28

...

recommendations finished on 1369000/1371980 queries. users per second: 8177.86

recommendations finished on 1370000/1371980 queries. users per second: 8177.26

recommendations finished on 1371000/1371980 queries. users per second: 8175.65

+-------------------------------+------------+-------------------+------+
|          customer_id          | article_id |       score       | rank |
+-------------------------------+------------+-------------------+------+
| 00000dbacae5abe5e23885899a... | 331474016  |        22.0       |  1   |
| 00000dbacae5abe5e23885899a... | 740299001  |        15.0       |  2   |
| 00000dbacae5abe5e23885899a... | 704760004  |        8.0        |  3   |
| 00000dbacae5abe5e23885899a... | 705500002  |        7.25       |  4   |
| 00000dbacae5abe5e23885899a... | 705671003  |        7.0        |  5   |
| 00000dbacae5abe5e23885899a... | 877607001  | 6.333333333333333 |  6   |
| 00000dbacae5abe5e23885899a... | 512568002  |        6.0        |  7   |
| 00000dbacae5abe5e23885899a... | 701784017  |        5.75       |  8   |
| 00000dbacae5abe5e23885899a... | 709248024  |        5.5        |  9   |
| 00000dbacae5abe5e23885899a... | 456834011  |        5.25       |  10  |
| 00000dbacae5abe5e23885899a... | 6906

Class                            : PopularityRecommender

Schema
------
User ID                          : customer_id
Item ID                          : article_id
Target                           : purchase_count
Additional observation features  : 0
User side features               : []
Item side features               : []

Statistics
----------
Number of observations           : 21845151
Number of users                  : 1325030
Number of items                  : 103491

Training summary
----------------
Training time                    : 1.1799

Model Parameters
----------------
Model class                      : PopularityRecommender

# 4 Collaborative Filtering Model

## 4.1 Cosine Similarity

In [ ]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

# name = 'cosine'
# target = 'purchase_dummy'
# cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

# name = 'cosine' 
# target = 'scaled_purchase_freq' 
# cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 21845151 observations with 1325030 users and 103491 items.

Data prepared in: 8.35614s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 1368000/1371980 queries. users per second: 6177.71

recommendations finished on 1369000/1371980 queries. users per second: 6178.52

recommendations finished on 1370000/1371980 queries. users per second: 6178.69

recommendations finished on 1371000/1371980 queries. users per second: 6177.83

+-------------------------------+------------+----------------------+------+
|          customer_id          | article_id |        score         | rank |
+-------------------------------+------------+----------------------+------+
| 00000dbacae5abe5e23885899a... | 568597006  | 0.01679846313264635  |  1   |
| 00000dbacae5abe5e23885899a... | 722483001  | 0.013888888888888888 |  2   |
| 00000dbacae5abe5e23885899a... | 893948002  | 0.013888888888888888 |  3   |
| 00000dbacae5abe5e23885899a... | 779781015  | 0.013183961311976114 |  4   |
| 00000dbacae5abe5e23885899a... | 408104022  | 0.01235415538152059  |  5   |
| 00000dbacae5abe5e23885899a... | 834596001  | 0.011340230703353882 |  6   |
| 00000dbacae5abe5e23885899a... | 771240002  | 0.011340230703353882 |  7   |
| 00000dbacae5abe5e23885899a... | 893948003  | 0.011340230703353882 |  8   |
| 00000dbacae5abe5e23885899a... | 774669002  | 0.01088879836930169  |  9   |
| 00000dbacae5abe5e23885899a... | 625548003  | 0.01052715712123447  |  10  |

## 4.2 Person Similarity

In [ ]:
# #1.
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)
# #2.
# name = 'pearson'
# target = 'purchase_dummy'
# pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)
#3.
# name = 'pearson'
# target = 'scaled_purchase_freq'
# pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)


Preparing data set.

Data has 21845151 observations with 1325030 users and 103491 items.

Data prepared in: 8.14927s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 1368000/1371980 queries. users per second: 6099.18

recommendations finished on 1369000/1371980 queries. users per second: 6098.49

recommendations finished on 1370000/1371980 queries. users per second: 6097.06

recommendations finished on 1371000/1371980 queries. users per second: 6093.78

+-------------------------------+------------+-------------------+------+
|          customer_id          | article_id |       score       | rank |
+-------------------------------+------------+-------------------+------+
| 00000dbacae5abe5e23885899a... | 331474016  |        22.0       |  1   |
| 00000dbacae5abe5e23885899a... | 740299001  |        15.0       |  2   |
| 00000dbacae5abe5e23885899a... | 704760004  |        8.0        |  3   |
| 00000dbacae5abe5e23885899a... | 705500002  |        7.25       |  4   |
| 00000dbacae5abe5e23885899a... | 705671003  |        7.0        |  5   |
| 00000dbacae5abe5e23885899a... | 877607001  | 6.333333333333334 |  6   |
| 00000dbacae5abe5e23885899a... | 512568002  |        6.0        |  7   |
| 00000dbacae5abe5e23885899a... | 701784017  |        5.75       |  8   |
| 00000dbacae5abe5e23885899a... | 709248024  |        5.5        |  9   |
| 00000dbacae5abe5e23885899a... | 456834011  |        5.25       |  10  |
| 00000dbacae5abe5e23885899a... | 8692

In [ ]:

# recom = cos.recommend(users=users_to_recommend, k=n_rec)
# df_rec = recom.to_dataframe()
# print(df_rec.shape)
# df_rec.head()

# 5. Model Evaluation


## 5.1

In [ ]:
# models_w_counts = [popularity, cos, pear]
# models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
# # models_w_norm = [pop_norm, cos_norm, pear_norm]
# names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
# names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
# # names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

# eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
# eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
# # eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

final_model = tc.item_similarity_recommender.create(tc.SFrame(customer_article), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_count', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec , new_observation_data=test_data)
recom.print_rows(n_display)

df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

Preparing data set.

Data has 27306439 observations with 1362281 users and 104547 items.

Data prepared in: 8.75597s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 1369000/1371980 queries. users per second: 7811.05

recommendations finished on 1370000/1371980 queries. users per second: 7809.33

recommendations finished on 1371000/1371980 queries. users per second: 7805.32

+-------------------------------+------------+----------------------+------+
|          customer_id          | article_id |        score         | rank |
+-------------------------------+------------+----------------------+------+
| 00000dbacae5abe5e23885899a... | 568597006  | 0.016577457126818206 |  1   |
| 00000dbacae5abe5e23885899a... | 779781015  | 0.01460350187201249  |  2   |
| 00000dbacae5abe5e23885899a... | 840676001  | 0.012405382959466232 |  3   |
| 00000dbacae5abe5e23885899a... | 793675001  | 0.011695905735618189 |  4   |
| 00000dbacae5abe5e23885899a... | 408104022  | 0.01102784746571591  |  5   |
| 00000dbacae5abe5e23885899a... | 580495001  | 0.01102784746571591  |  6   |
| 00000dbacae5abe5e23885899a... | 722575001  | 0.010579344473387065 |  7   |
| 00000dbacae5abe5e23885899a... | 625548003  | 0.010522905148957906 |  8   |
| 00000dbacae5abe5e23885899a... | 568601007  | 0.01051250884407445  |  9   |
| 00000dbacae5abe5e23885899a... | 858856005  | 0.010381140206989489 |  10  |

,customer_id,article_id,score,rank
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568597006,0.016577,1
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,779781015,0.014604,2
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,840676001,0.012405,3
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,793675001,0.011696,4
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,408104022,0.011028,5


# 6. Data Submission

In [ ]:
# submission_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
# submission_df["prediction"] = " "
# submission_df["prediction"] = submission_df["customer_id"].map(customer_article)
# submission_df["prediction"] = submission_df["prediction"].apply(lambda x: x if isinstance(x, list) else [])
# submission_df["prediction"] = submission_df["prediction"].apply(lambda x: x[:12] + popular_articles[:12-len(x)])
# submission_df.head()
# submission_df["prediction"] = submission_df["prediction"].apply(lambda x:" ".join(map(str,x)))
result = df_rec.groupby("customer_id")["article_id"].apply(list)

# # submission_df
# submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df = pd.DataFrame({'customer_id':result.index, 'prediction':result.values})
submission_df["prediction"] = submission_df["prediction"].apply(lambda x:" ".join(map(str,x)))
submission_df
submission_df.to_csv('/content/drive/My Drive/Colab Notebooks/567_hw/output/submission1.csv', index=False)
